In [1]:
# Connect to GDrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pickle
import sys
from math import floor
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import torch
import torch.nn as nn
import sys
from torch import nn
from torch.utils.data import ConcatDataset, DataLoader, TensorDataset
import random
import pandas as pd

In [10]:
train = pd.read_parquet("/content/drive/MyDrive/MIiF/train_data/train_data_0.pq")

In [11]:
len(train)

1974724

In [3]:
from drive.MyDrive.MIiF.utils import train_epoch, eval_model, inference
from drive.MyDrive.MIiF.models import TransactionsRnn, TransactionsRnn2
from drive.MyDrive.MIiF.preprocess import features, embedding_projection, read_parquet_dataset_from_local, batches_generator, pad_sequence, truncate, transform_transactions_to_sequences, create_padded_buckets, create_buckets_from_transactions

In [20]:
new_tr = train.groupby("id").apply(lambda x: len(x.values)).reset_index().rename(columns={0: 'len'})

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [5]:
TRAIN_TRANSACTIONS_PATH = 'drive/MyDrive/MIiF/train_data'
TRAIN_TARGET_PATH = 'drive/MyDrive/MIiF/train_target.csv'
#MODEL_PATH = 'alpha_task_2/models'
SOL_PATH = 'drive/MyDrive/MIiF/sols'
TRAIN_BACKETS_PATH = 'drive/MyDrive/MIiF/train_buckets'

In [6]:
train_targ = pd.read_csv(TRAIN_TARGET_PATH)

In [8]:
train_targ

,id,flag
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
2999995,2999995,0
2999996,2999996,0
2999997,2999997,0
2999998,2999998,0


In [ ]:
create_buckets_from_transactions(TRAIN_TRANSACTIONS_PATH,
                                  save_to_path=TRAIN_BACKETS_PATH, frame_with_ids=train_targ,
                                  num_parts_to_preprocess_at_once=1, num_parts_total=12, has_target=True)

Transforming transactions data:   0%|          | 0/12 [00:00<?, ?it/s]

Reading chunks:

drive/MyDrive/MIiF/train_data/train_data_0.pq


Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting buckets:   0%|          | 0/250000 [00:00<?, ?it/s]

Reading chunks:

drive/MyDrive/MIiF/train_data/train_data_1.pq


Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
model1 = TransactionsRnn(,embedding_projections,rnn_units = 1024*4,top_classifier_units=128)

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 3.5 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier, Pool, metrics

In [ ]:
cat_features = list(train.columns)

In [ ]:
# Агрегаты для бустинга через pivot
def __amnt_pivot_table_by_column_as_frame(frame, column, agg_funcs=None) -> pd.DataFrame:
    if agg_funcs is None:
        agg_funcs = ['mean', 'count']
    aggs = pd.pivot_table(frame, values='amnt',
                          index=['app_id'], columns=[column],
                          aggfunc={'amnt': agg_funcs},
                          fill_value=0.0)
    aggs.columns = [f'{col[0]}_{column}_{col[1]}' for col in aggs.columns.values]
    return aggs

In [ ]:
# Функция для расчёта фичей для модели бустинга
def preprocess(train_df):
  good_cols = joblib.load('alpha_task_2/models/good_cols')
  train_df['amnt_rub'] = np.exp(train_df['amnt']*17.8209)-1
  cols1 = ['amnt', 'amnt_rub', 'hour_diff', 'days_before']
  feat0 = pd.DataFrame()
  for i in cols1:
    if i == 'amnt' or i == 'amnt_rub':
      amnt_feat = pd.DataFrame(train_df.groupby(['app_id'])[i].agg(['count', 'mean', 'max', 'min', 'sum', 'median', 'std']))
      amnt_feat = amnt_feat.merge(pd.DataFrame(train_df[train_df.amnt==0].groupby(['app_id'])[i].agg(['count'])), on = 'app_id', how='left')
      amnt_feat.columns=[f'{i}_count', f'{i}_mean', f'{i}_max', f'{i}_min', f'{i}_sum', f'{i}_zero_count', f'{i}_median', f'{i}_std']
      amnt_feat[f'{i}_zero_share'] =  amnt_feat[f'{i}_zero_count'] / amnt_feat[f'{i}_count']
    else:
      amnt_feat = pd.DataFrame(train_df.groupby(['app_id'])[i].agg(['mean', 'max', 'min', 'sum', 'median', 'std']))
      amnt_feat.columns=[ f'{i}_mean', f'{i}_max', f'{i}_min', f'{i}_sum', f'{i}_median', f'{i}_std']
    feat0 = pd.concat([amnt_feat, feat0], axis=1)

  cols2 = ['currency', 'operation_kind', 'card_type', 'operation_type', 'operation_type_group',
           'ecommerce_flag',	'payment_system',  'mcc',
           		'city', 'mcc_category', 'day_of_week', 'hour', 'weekofyear']
  feat = pd.DataFrame()
  for i in cols2:
    if i in ['card_type', 'mcc', 'mcc_category', 'hour', 'weekofyear']:
      currency_feat = pd.DataFrame(train_df.groupby(['app_id'])[i]\
                              .agg([lambda x: x.nunique(), lambda x: stats.mode(x)[0][0],
                                    lambda x: stats.mode(x)[1][0] / x.count()]))
      currency_feat.columns=[f'{i}_nunique', f'{i}_most_pop', f'{i}_most_pop_share']
    else:
      currency_feat = pd.DataFrame(train_df.groupby(['app_id'])[i]\
                              .agg([lambda x: stats.mode(x)[0][0],
                                    lambda x: stats.mode(x)[1][0] / x.count()]))
      currency_feat.columns=[f'{i}_most_pop', f'{i}_most_pop_share']
    feat = pd.concat([currency_feat, feat], axis=1)

  pivot_tables = []
  for col in cols2[:-1]:
      pivot_tables.append(__amnt_pivot_table_by_column_as_frame(train_df, column=col))
  pivot_tables = pd.concat(pivot_tables, axis=1)
  all_feat = feat.merge(feat0, left_index=True, right_index=True).merge(pivot_tables, left_index=True, right_index=True)
  our_cols = []
  for i in good_cols:
    if i in all_feat.columns:
      our_cols.append(i)
  for i in all_feat.columns:
    if 'rub' in i:
      our_cols.append(i)
  return all_feat[our_cols]

In [ ]:
params = {
    # 'iterations': 500,
    # 'learning_rate': 0.1,
    'depth': 6,
    'loss_function': 'Logloss',
    'random_seed': 42,
    #'use_best_model': True,
    #'eval_metric': metrics.Accuracy(),
    "auto_class_weights": "Balanced"
}

model = CatBoostClassifier(**params)

In [ ]:
for i in range(12):
  tr_data =